In [3]:
import sklearn
import lightgbm as lgb
import pandas as pd 
from tqdm.autonotebook import tqdm
import numpy as np
import gc
data_dir = './data/'

In [4]:
labels = pd.DataFrame()
trains = pd.DataFrame()
tests = pd.DataFrame()
gc.collect()
for i in tqdm(range(1, 6)):
    trains = trains.append(pd.read_csv(data_dir+'train%d.csv'%i, header=None))
    labels = labels.append(pd.read_csv(data_dir+'label%d.csv'%i, header=None))
    tests = tests.append(pd.read_csv(data_dir+'test%i.csv'%i, header=None))
tests = tests.append(pd.read_csv(data_dir+'test6.csv', header=None))

In [5]:
import sklearn.model_selection
def cv(x, y, model, params={}, splits=5):
    clf = model(**params)
    kfold = sklearn.model_selection.KFold(splits, shuffle=True)
    cv_score = sklearn.model_selection.cross_validate(clf, x, y, cv=kfold, scoring={
        'mse': 'neg_mean_squared_error',
        'r2': 'r2'
    }, return_train_score=True, verbose=5)
    return cv_score

In [6]:
def run_cross_validation(x, y, model, params={}):
    cv_result = cv(x, y, model, params=params, splits=3)
    for scorer, score in cv_result.items():
        print('%s: %s' % (scorer, score))
        print('Average %s: %f' % (scorer, score.mean()))

In [7]:
params = {
        'boosting_type': 'gbdt',
    
#         'n_estimators': 3000,
#         "num_leaves": 800,
#         "subsample_for_bin": 100000,
#         'max_bin': 1500,
    
        'n_estimators': 600,
        "num_leaves": 80,
        "subsample_for_bin": 50000,
        'max_bin': 1000,
    
        'feature_fraction': 0.4,
        'bagging_fraction': 0.8,
        'bagging_freq': 5,
        'reg_alpha': 4,
        'reg_lambda': 8,
        
        "max_depth": -1,
        'min_child_samples': 100,
        'learning_rate': 0.05,
        'n_jobs': -1,
        'verbose': 5
}

In [ ]:
run_cross_validation(trains, labels, lgb.LGBMRegressor, params)

In [8]:
model = lgb.LGBMRegressor(**params)
model.fit(trains, labels)
result = model.predict(tests)

In [9]:
res_frame = pd.DataFrame()
res_frame["Id"] = [e for e in range(1, tests.shape[0]+1)]

In [10]:
res_frame["Predicted"] = result
res_frame.to_csv("lgb.csv", index=False)